In [3]:
import logging

from config import UserConfiguration, LogConfiguration, TorchConfiguration, TokenizerConfiguration, SystemConfiguration

from os_environment_manager import OSEnvironmentManager
from package_path_manager import PackagePathManager
from model_manager import ModelManager
from system_monitor import SystemMonitor

from tokenizer import Tokenizer
from data_manager import DataManager


# TODO: These should be picked up from command line
NET_ID = "vgn2004"
ENV = "pre_prod"
NUM_WORKERS = 8
MAX_TOKENS = 512
MODEL_NAME = "facebook/opt-125m"
DATASET_NAME = "NIH_ExPORTER_awarded_grant_text"
BATCH_SIZE = 64

# Constants
OS_ENV_DICT = {
    "CUDA_VISIBLE_DEVICES": 0,
    "TRANSFORMERS_NO_ADVISORY_WARNINGS": "true",
    "TORCHDYNAMO_DISABLE": 1
}

In [4]:
# Configure the logger, needed for initial utilization checks
LogConfiguration.setup_logging()
logger = logging.getLogger(__name__)

# Get initial RAM and GPU utilization
monitor = SystemMonitor()
logger.info(f"RAM Usage: {monitor.get_ram_usage()} MB")
logger.info(f"GPU Utilization: {monitor.get_gpu_utilization()} MB")

# Configurations

# Setup folder/file path related configurations
user_config = UserConfiguration(net_id=NET_ID, env=ENV)
system_config = SystemConfiguration(num_workers=NUM_WORKERS)
tokenizer_config = TokenizerConfiguration(max_tokens=MAX_TOKENS)
torch_config = TorchConfiguration()
torch_config.commit()

# System configurations

# Add Python packages to sys path
package_path_manager = PackagePathManager(user_config)
package_path_manager.add_package_paths_to_system()

# Add environment variables to OS env
os_env_manager = OSEnvironmentManager()
os_env_manager.update_from_dict(OS_ENV_DICT)

# Tokenization
tokenizer = Tokenizer(user_config, tokenizer_config)
tokenizer.load_for_model(MODEL_NAME)

# Datasets
data_manager = DataManager(tokenizer, user_config, system_config)
data_manager.dataset_name = DATASET_NAME

# Tokenize dataset from scratch (skipped)
# data_manager.create_dataset_from_jsonl_zst_file(name=DATASET_NAME,
#                                                 jsonl_zst_file_path="E:\\NIH_ExPORTER_awarded_grant_text.jsonl.zst",
#                                                 save_to_disk=False)
# data_manager.create_tokenized_dataset(save_to_disk=False)
# training_dataset, validation_dataset = data_manager.fetch_train_validation_split(save_to_disk=False)

# Load from disk
training_dataset, validation_dataset = data_manager.fetch_train_validation_split_from_disk()

# Dataloaders
training_dataloader, validation_dataloader = data_manager.fetch_dataloaders(
    training_dataset=training_dataset,
    validation_dataset=validation_dataset,
    batch_size=BATCH_SIZE
)

# Model
model_manager = ModelManager(system_config)
model_manager.load(MODEL_NAME)
logger.info(model_manager.model)

INFO: At 01:04 AM 54s, from <ipython-input-4-7792d595e683>:<module>:7 => RAM Usage: 386.85546875 MB
INFO: At 01:04 AM 54s, from <ipython-input-4-7792d595e683>:<module>:7 => RAM Usage: 386.85546875 MB
INFO: At 01:04 AM 54s, from <ipython-input-4-7792d595e683>:<module>:8 => GPU Utilization: 1254 MB
INFO: At 01:04 AM 54s, from <ipython-input-4-7792d595e683>:<module>:8 => GPU Utilization: 1254 MB
INFO: At 01:04 AM 54s, from torch_configuration.py:commit:14 => Flash attention is enabled!
INFO: At 01:04 AM 54s, from torch_configuration.py:commit:14 => Flash attention is enabled!
/share/apps/python/3.8.6/intel/lib/python3.8/site-packages/setuptools/distutils_patch.py:25: UserWarning: Distutils was imported before Setuptools. This usage is discouraged and may exhibit undesirable behaviors or errors. Please use Setuptools' objects directly or at least import Setuptools first.
  warnings.warn(
INFO: At 01:04 AM 55s, from instantiator.py:<module>:21 => Created a temporary directory at /state/part